In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import flopy
import pyemu

In [ ]:
org_d = os.path.join("..", "models", "synthetic-valley-base-annual")
assert os.path.exists(org_d)

In [ ]:
new_d = org_d + "-optwell"
if os.path.exists(new_d):
    shutil.rmtree(new_d)

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=org_d)

In [ ]:
gwf = sim.get_model()

In [ ]:
wel = gwf.pwell.stress_period_data.array
upper_flux = wel[1]["q"][:2].sum()
lower_flux = wel[1]["q"][2:].sum()

In [ ]:
nrow, ncol = gwf.dis.nrow.data, gwf.dis.ncol.data

In [ ]:
karr = gwf.npf.k.array

In [ ]:
kvals, ivals, jvals = [], [], []
for i in range(2, nrow - 2, 5):
    for j in range(2, ncol - 2, 5):
        if karr[0, i, j] > 1000:
            continue
        # print(karr[0,i,j])
        for k in [1, 4]:
            kvals.append(k)
            ivals.append(i)
            jvals.append(j)

In [ ]:
upper_flux_per_wel = upper_flux / (len(kvals) / 2)
lower_flux_per_wel = lower_flux / (len(kvals) / 2)

In [ ]:
df = pd.DataFrame({"k": kvals, "i": ivals, "j": jvals})
df

In [ ]:
df["q"] = np.nan

In [ ]:
df.loc[df.k == 1, "q"] = upper_flux_per_wel
df.loc[df.k == 4, "q"] = lower_flux_per_wel
df

In [ ]:
gwf.remove_package("pwel")

In [ ]:
gwf.remove_package("prediction")

In [ ]:
spd = {}
for kper in range(sim.tdis.nper.data):
    spd[kper] = df.to_records(index=False)
df["q"] = 0.0
spd[0] = df

In [ ]:
flopy.mf6.ModflowGwfwel(gwf, stress_period_data=spd)

In [ ]:
sim.set_all_data_external()

In [ ]:
sim.simulation_data.max_columns_of_data = nrow

In [ ]:
sim.set_sim_path(new_d)

In [ ]:
sim.write_simulation()

In [ ]:
pyemu.os_utils.run_sp("mf6", cwd=new_d)